## Policy Gradient 
----

Ref: [Teaching material at UC Berkeley (PDF)](http://rail.eecs.berkeley.edu/deeprlcourse-fa17/f17docs/lecture_4_policy_gradient.pdf)

Solve easy maze by policy gradient with classic REINFORCE algorithm

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#convert params into probabilities
def softmax(theta):
    beta = 1
    m, n = theta.shape
    pi = np.zeros((m, n)) # m:status, n:action
    exp_theta = np.exp(beta * theta)
    
    for i in range(m):
        pi[i, :] = exp_theta[i, :] / np.nansum(exp_theta[i, :])
        
    pi = np.nan_to_num(pi)
    
    return pi

def get_action_and_next_s(pi, s):
    direction = ['up', 'right', 'down', 'left']
    
    next_direction = np.random.choice(direction, p=pi[s, :])
    
    if next_direction == 'up':
        return [0, s - 3]
    elif next_direction == 'right':
        return [1, s + 1]
    elif next_direction == 'down':
        return [2, s + 3]
    else:
        return [3, s - 1]
    
def goal_maze(pi):
    s = 0
    s_a_history = [[0, np.nan]]
    
    while True:
        [action, s] = get_action_and_next_s(pi, s)
        s_a_history[-1][1] = action
        
        s_a_history.append([s, np.nan])
        
        if s is 8:
            break
    return s_a_history

### How to update policy parameter $\theta$
$$
\theta_{n+1} = \theta_n + \eta \cdot \nabla J(\theta) \\
\nabla J(\theta) = \mathbb{E}\left[Q(s,a)
    \frac{\partial\log\pi_{S_t,A_t}}{\partial \theta}\right]
$$

<!---
$\pi_{s,a} \equiv \pi(A_t = a|S_t = s)$, and $\theta_{s,a}$ is defined similarly.
--->

Use REINFORCE algorithm, which approximates $Q$ value by $R$ and estimate the gradient of $J$

$$
\nabla J(\theta) \approx \frac{1}{T}
    \left(\sum_t^TR_t - b\right)\left(\sum_t^T\frac{\partial\log\pi_{S_t,A_t}}{\partial\theta}\right)
$$

This formula intends to increase the probability of choosing an action 
by which an agent gained more total reward in shorter period.<br>

Now an agent reaches goal S8 and obtain reward in every trial, we use $\sum_tR_t=1$ and baseline constant $b = 0$
for simplification.

Also, now $\pi_{s,a} = {\rm SM}(\theta_{s,a})$ therefore 
$$
\frac{\partial \pi_{S_t,A_t}}{\partial \theta_{s,a}} =
    \begin{cases}
        0 & {\rm for}& S_t \neq s\\
        \pi_{S_t,A_t}(\delta_{A_t, a} - \pi_{S_t, a}) & {\rm for}& S_t = s \\
    \end{cases}
$$


So we can rewrite $\nabla J(\theta_{s,a})$ as following:

$$
\nabla J(\theta_{s,a}) \approx \frac{1}{T}\cdot 1 \cdot \left(\sum_{t (if )}^T \frac{\partial\log\pi_{S_t,A_t}}{\partial\pi_{S_t, A_t}}\frac{\partial\pi_{S_t, A_t}}{\partial\theta_{s, a}}\right) \\
    = \frac{1}{T}\sum_t^T \delta_{S_t, s}(\delta_{A_t, a}- \pi_{s, a})\\
    = \frac{N_{s,a} - N_s \cdot \pi_{s, a}}{T}
$$

In above equation, $N_i$ means how many times an agent experience state $i$,<br> 
and $N_{i,j}$ means how many times an agent experience state $i$ and then take action $j$.

In [2]:
def update_theta(theta, pi, s_a_history, lr=1.0):
    T = len(s_a_history)
    
    m, n = theta.shape # m:status n:action
    delta_theta = np.zeros(theta.shape)
    
    for i in range(m):
        for j in range(n):
            if not np.isnan(theta[i, j]):
                N_i = len([sa for sa in s_a_history if sa[0] is i]) #how many times in s[i]
                N_ij = len([sa for sa in s_a_history if sa == [i, j]]) #how many times in s[i] and take a[j]
                delta_theta[i, j] = (N_ij - pi[i,j] * N_i) / T
    
    return theta + lr * delta_theta

In [3]:
#initial params
theta = np.array([[np.nan, 1, 1, np.nan],  #s0
                   [np.nan, 1, np.nan, 1], #s1
                   [np.nan, np.nan, 1, 1], #s2
                   [1, 1, 1, np.nan],
                   [np.nan, np.nan, 1, 1],
                   [1, np.nan, np.nan, np.nan],
                   [1, np.nan, np.nan, np.nan],
                   [1, 1, np.nan, np.nan], #s7
                   ])

#initial probabilities
pi = softmax(theta)

#learning
stop_epsilon = 1e-4
while True:
    s_a_history = goal_maze(pi)
    theta = update_theta(theta, pi, s_a_history, lr=20)
    new_pi = softmax(theta)
    
    if np.sum(np.abs(new_pi - pi)) < stop_epsilon:
        break
        
    pi = new_pi
print('Step:', len(s_a_history)-1)

Step: 4


In [4]:
#draw result
from matplotlib import animation
from IPython.display import HTML

fig = plt.figure(figsize=(5,5))
ax = plt.gca()

#draw walls
plt.plot([1,1], [0,1], color='r', linewidth=2)
plt.plot([1,2], [2,2], color='r', linewidth=2)
plt.plot([2,2], [2,1], color='r', linewidth=2)
plt.plot([2,3], [1,1], color='r', linewidth=2)

#write status
plt.text(0.5, 2.5, 'S0', size=14, ha='center')
plt.text(1.5, 2.5, 'S1', size=14, ha='center')
plt.text(2.5, 2.5, 'S2', size=14, ha='center')
plt.text(0.5, 1.5, 'S3', size=14, ha='center')
plt.text(1.5, 1.5, 'S4', size=14, ha='center')
plt.text(2.5, 1.5, 'S5', size=14, ha='center')
plt.text(0.5, 0.5, 'S6', size=14, ha='center')
plt.text(1.5, 0.5, 'S7', size=14, ha='center')
plt.text(2.5, 0.5, 'S8', size=14, ha='center')
plt.text(0.5, 2.3, 'START', size=14, ha='center')
plt.text(2.5, 0.3, 'GOAL', size=14, ha='center')

ax.set_xlim(0,3)
ax.set_ylim(0,3)

plt.tick_params(axis='both', which='both', bottom=False, top=False,
                labelbottom=False, right=False, left=False, labelleft=False)
line, = ax.plot([0.5], [2.5], marker='o', color='g', markersize=60)
plt.close(fig)

def init():
    line.set_data([], [])
    return line

def animate(i):
    state = s_a_history[i][0]
    x = state % 3 + 0.5
    y = 2.5 - int(state / 3)
    line.set_data(x, y)
    return line

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(s_a_history), interval=300, repeat=False)
HTML(anim.to_jshtml())